# CONTAINER AND REQUEST USAGE GUIDE

In addition to invoking methods directly, as detailed in the `general_workflow.md` guide, you can perform various actions or calculations through requests. For examples of this approach, see `tests/risk_models/risk_models_AK_test.py` and `tests/risk_models/risk_models_test.py`.

## Process

1. **Create a Container**

   First, create a `Container` object and configure it by overriding its default providers with custom ones. Here’s an example:

   ```python
   # Define custom factories for hazard and vulnerability models
   class TestHazardModelFactory(HazardModelFactory):
       def hazard_model(self, interpolation: str = "floor", provider_max_requests: Dict[str, int] = ...):
           return ZarrHazardModel(
               source_paths=get_default_source_paths(), reader=reader
           )

   class TestVulnerabilityModelFactory(VulnerabilityModelsFactory):
       def vulnerability_models(self):
           return DictBasedVulnerabilityModels(
               {
                   ThermalPowerGeneratingAsset: [
                       ThermalPowerGenerationAqueductWaterStressModel()
                   ]
               }
           )

   # Register custom providers in the container
   container.override_providers(
       hazard_model_factory=providers.Factory(TestHazardModelFactory)
   )
   container.override_providers(
       config=providers.Configuration(default={"zarr_sources": ["embedded"]})
   )
   container.override_providers(inventory_reader=ZarrReader())
   container.override_providers(zarr_reader=ZarrReader())
   container.override_providers(
       vulnerability_models_factory=providers.Factory(TestVulnerabilityModelFactory)
   )

    ```   
    You can include any list of vulnerability models in the configuration. If none are provided, default models will be used.


2. **Create a Requester**

    After setting up the container, call ``container.requester()`` to obtain an instance of ``Requester``. This object includes the following attributes configured from the container:
   ``hazard_model_factory: HazardModelFactory, vulnerability_models_factory: VulnerabilityModelsFactory, inventory: Inventory, inventory_reader: InventoryReader,   reader: ZarrReader, colormaps: Colormaps`` and a ``measures_factory: RiskMeasuresFactory``
   

3. **Call the Method and Obtain a Response**

    The `Requester` class has a main method that calls different methods based on the `request_id` provided.

    Here is an example of how to call a method using the `get` method:

    ```python
    res = requester.get(request_id="get_asset_impact", request_dict=request_dict)
    ```

    You can assign the following values to `request_id`:

    - `get_hazard_data`: Returns intensity curves for the selected hazards, years, and scenarios.
    - `get_hazard_availability`: Returns the hazards stored in the inventory.
    - `get_hazard_description`: Returns the description assigned to a specific hazard.
    - `get_asset_exposure`: Calculates the exposure of a given asset for a hazard, exposure measure, scenario, and year.
    - `get_asset_impact`: Returns risk measures or impacts based on parameters provided in `request_dict`.
    - `get_example_portfolios`: Returns a JSON with assets and their respective details such as class, type, location, latitude, and longitude.

    The structure of `request_dict` depends on the method you are calling. For example, for the `get_asset_impact` method, `request_dict` might look like this:

    ```python
    def create_assets_json(assets: Sequence[ThermalPowerGeneratingAsset]):
        assets_dict = {
            "items": [
                {
                    "asset_class": type(asset).__name__,
                    "type": asset.type,
                    "location": asset.location,
                    "longitude": asset.longitude,
                    "latitude": asset.latitude,
                }
                for asset in assets
            ],
        }
        return assets_dict

    request_dict = {
        "assets": create_assets_json(assets=assets),
        "include_asset_level": False,
        "include_measures": True,
        "include_calc_details": False,
        "model_kind": ModelKind.STRESS_TEST,
        "years": years,
        "scenarios": scenarios,
    }
    ```

    Finally, the ``get`` method calls the appropriate methods corresponding to the ``request_id`` with the necessary parameters and returns the response as a JSON object with the result.